<a href="https://colab.research.google.com/github/helenross636/magic_season/blob/main/%23Task_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
import time

Загружаем модель на 30 нейронов и тестовую выборку.

In [5]:
#model = keras.models.load_model('/content/model1.h5')
model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/models/model1.h5')

(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Преобразование размерности изображений
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
# Нормализация данных
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

11501568/11490434 [==============================] - 0s 0us/step


In [6]:
#загружаем веса модели на 30
w1 = np.transpose(model.layers[2].kernel)
(y1,z1) = w1.shape
print(w1.shape)
bias1 =  np.transpose(model.layers[2].bias)
print(bias1.shape)
w2 = (model.layers[3].kernel)
bias2 =  (model.layers[3].bias)
(y2,z2) = w2.shape
print(w2.shape)
print(bias2.shape)


(30, 784)
(30,)
(30, 10)
(10,)


In [7]:
#создаем матрицу весов в 50 раз больше
#input layer[2]
w50_1 = np.zeros((y1*50,z1)) #kernel
print(w50_1.shape)
b50_1 = np.zeros(len(bias1)*50)
print(b50_1.shape)
#output layer[3]
w50_2 = np.zeros((y2*50,z2)) #kernel
print(w50_2.shape)
b50_2 = np.zeros(10)
print(b50_2.shape)


(1500, 784)
(1500,)
(1500, 10)
(10,)


In [8]:
#создаем модель в 50 раз больше
inputs = layers.Input(shape = (784,), name = "in_layer")
x = layers.Flatten()(inputs)
x = layers.Dense(y1*50, activation="relu", name = "hidden_layer")(x)
outputs = layers.Dense(10, activation="softmax", name = "out_layer")(x)
model_50 = tf.keras.Model(inputs = inputs, outputs = outputs)
model_50.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

print(model_50.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 in_layer (InputLayer)       [(None, 784)]             0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 hidden_layer (Dense)        (None, 1500)              1177500   
                                                                 
 out_layer (Dense)           (None, 10)                15010     
                                                                 
Total params: 1,192,510
Trainable params: 1,192,510
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#w50 = np.transpose(model_50.layers[2].kernel)
#(yy,zz) = w50.shape
#print(w50[0])

In [9]:
#заполняем веса для бОльшей модели
for i in range(y1*50):
  if i<30: j = i
  else: j = i%30
  w50_1[i] = w1[j]

for i in range(y2*50):
  if i<30: j = i
  else: j = i%30
  w50_2[i] = w2[j]

for i in range(len(bias1)*50):
  if i<30: j = i
  else: j = i%30
  b50_1[i] = bias1[j]

b50_2 = bias2
#делим выходные веса на 50
w50_2 = w50_2/50

In [ ]:
#маленькая проверочка
print(w50[0])
print('------------------------------------------------------------------------------')
print(w50[30])

In [10]:
#заполняем модель
#model_50.set_weights(w50)
print('input layer kernel shape: ',model_50.layers[2].get_weights()[0].shape)
print('input layer bias shape: ',model_50.layers[2].get_weights()[1].shape)
w50_1T = np.transpose(w50_1)
b50_1T = np.transpose(b50_1)
print('w50_1T shape:', w50_1T.shape)
print('b50_1T shape:', b50_1T.shape)
model_50.layers[2].set_weights([w50_1T, b50_1T])
print('output layer kernel shape: ',model_50.layers[3].get_weights()[0].shape)
print('output layer bias shape: ',model_50.layers[3].get_weights()[1].shape)
w50_2T = (w50_2)
b50_2T = np.transpose(b50_2)
print('w50_2T shape:', w50_2T.shape)
print('b50_2T shape:', b50_2T.shape)
model_50.layers[3].set_weights([w50_2T, b50_2T])


input layer kernel shape:  (784, 1500)
input layer bias shape:  (1500,)
w50_1T shape: (784, 1500)
b50_1T shape: (1500,)
output layer kernel shape:  (1500, 10)
output layer bias shape:  (10,)
w50_2T shape: (1500, 10)
b50_2T shape: (10,)


In [11]:
#смотрим результат
scores = model_50.evaluate(X_test, Y_test, verbose=0);
print("РАЗМНОЖЕННАЯ СЕТЬ. Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))  

РАЗМНОЖЕННАЯ СЕТЬ. Точность работы на тестовых данных: 96.97%


In [ ]:
model2 = keras.models.load_model('/content/model2.h5')
model3 = keras.models.load_model('/content/model3.h5')
model4 = keras.models.load_model('/content/model4.h5')

In [ ]:
scores = model.evaluate(X_test, Y_test, verbose=0);
print("ОРИГИНАЛЬНАЯ МОДЕЛЬ (30). Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))  
print('-----------------------------------------------------------------------------')
scores = model2.evaluate(X_test, Y_test, verbose=0);
print("model2 (1500). Точность работы на тестовых данных: %.2f%%" % (scores[1]*100)) 
print('-----------------------------------------------------------------------------')
scores = model3.evaluate(X_test, Y_test, verbose=0);
print("model3 (5000). Точность работы на тестовых данных: %.2f%%" % (scores[1]*100)) 
print('-----------------------------------------------------------------------------')
scores = model4.evaluate(X_test, Y_test, verbose=0);
print("model4 (10000). Точность работы на тестовых данных: %.2f%%" % (scores[1]*100)) 
print('-----------------------------------------------------------------------------')


ОРИГИНАЛЬНАЯ МОДЕЛЬ (30). Точность работы на тестовых данных: 96.71%
-----------------------------------------------------------------------------
model2 (1500). Точность работы на тестовых данных: 98.30%
-----------------------------------------------------------------------------
model3 (5000). Точность работы на тестовых данных: 98.30%
-----------------------------------------------------------------------------
model4 (10000). Точность работы на тестовых данных: 98.38%
-----------------------------------------------------------------------------
